In [9]:
#set project path from config.yaml
import pathlib
import yaml

with open("config.yaml", "r") as f:
   config = yaml.safe_load(f)

project_root = pathlib.Path(config["project"]["root_path"])

print("Current project path: ", project_root)

Current project path:  D:\Data Repositories\Coughvid


In [10]:
dset_wav_path = project_root / "audio_wav_fulldset"
print(type(dset_wav_path))

<class 'pathlib.WindowsPath'>


In [11]:
import os
import matplotlib.pyplot as plt
import numpy as np
from pyAudioAnalysis import ShortTermFeatures
from pyAudioAnalysis import audioBasicIO
import matplotlib.pyplot as plt
import pandas as pd
import random


wav_paths = list(dset_wav_path.glob("*.wav"))

random_wav = random.choice(wav_paths)
wav_path = os.path.join(dset_wav_path, random_wav)

print(wav_path)

[Fs, x] = audioBasicIO.read_audio_file(wav_path)


def extract_features(path): #return 1x34 df with the avgs of the features of mid_term_extraction
    
    [Fs, x] = audioBasicIO.read_audio_file(str(path))

    #Check for empty audio
    if x.size == 0:
        return None  # signal invalid file

    F, f_names = ShortTermFeatures.feature_extraction(x, Fs, 0.050*Fs, 0.025*Fs, deltas = False) #using SHORT term extraction due to small length of audio files
    
    df = pd.DataFrame(F.T, columns=f_names) #nx34
    #compute averages over time to get fixed length vector
    df_avg = df.mean(axis=0).to_frame().T #1x34

    return df_avg





D:\Data Repositories\Coughvid\audio_wav_fulldset\d01e80ff-c38e-4eea-8965-892086212f85.wav


In [16]:
from tqdm import tqdm

invalid_files_counter = 0
invalid_files_paths = []

final_df = pd.DataFrame()


# Limit to first 2000 samples
sample_limit = 2000
wav_paths_subset = wav_paths[:sample_limit]

for wav_path in tqdm(wav_paths_subset, desc="Processing dset"):
    
    #print(wav_path)
    wav_path = os.path.join(dset_wav_path, random_wav)
    wav_avg = extract_features(wav_path)
    if wav_avg is None:
        invalid_files_counter += 1
        invalid_files_paths.append(wav_path)
        #print(f"Skipped empty file: {wav_file}")
        continue
    #append file to final df if ok 
    final_df = pd.concat([final_df, wav_avg], ignore_index=True)
    #print(wav_avg)
    

print("skipped files: ",invalid_files_counter)


Processing dset: 100%|██████████| 2000/2000 [12:27<00:00,  2.68it/s]

skipped files:  0


In [19]:
print(final_df.shape)
final_df.to_csv("toy_dset_2k.csv", index = False)

(2000, 34)
